In [7]:
import json
import re

# Function to parse 'other_tags' string
def parse_other_tags(tags):
    # Create a dictionary from the 'other_tags' string
    tag_dict = {}
    tag_pairs = re.findall(r'"([^"]+)"=>"(.*?)"', tags)
    for key, value in tag_pairs:
        tag_dict[key] = value
    return tag_dict

# Function to load GeoJSON from a file
def load_geojson(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Function to save GeoJSON to a file
def save_geojson(data, file_path):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

# Main function to filter house points and create new GeoJSON
def extract_house_points(input_file, output_file):
    # Load the GeoJSON data from the input file
    geojson_data = load_geojson(input_file)

    house_points = []
    for feature in geojson_data['features']:
        properties = feature['properties']
        
        # Check if the feature has 'other_tags' and contains both housenumber and street
        if 'other_tags' in properties:
            tags = properties['other_tags']
            tag_dict = parse_other_tags(tags)
            
            if 'addr:housenumber' in tag_dict and 'addr:street' in tag_dict:
                # Create a new feature for the house point
                house_point = {
                    "type": "Feature",
                    "properties": {
                        "osm_id": properties["osm_id"],
                        "house_number": tag_dict['addr:housenumber'],
                        "street_name": tag_dict['addr:street']
                    },
                    "geometry": feature["geometry"]
                }
                house_points.append(house_point)

    # Create a new GeoJSON structure
    house_geojson = {
        "type": "FeatureCollection",
        "features": house_points
    }

    # Save the resulting GeoJSON to the output file
    save_geojson(house_geojson, output_file)

    print(f"Processed GeoJSON saved to {output_file}")

# Example usage
input_file = 'points.geojson'  # Path to your input GeoJSON file
output_file = 'OSM.geojson'  # Path where the output will be saved

# Run the function to process the file
extract_house_points(input_file, output_file)


Processed GeoJSON saved to OSM.geojson
